# Importación de Base de Datos

## Librerías

### Procesamiento de datos

In [1]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objs as go
import numpy as np
from prophet.diagnostics import cross_validation, performance_metrics
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Graficos

In [19]:
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

### Rutas de los archivos

In [2]:
import routes

## Creación de dataframes

### Importación de los datos crudos

In [3]:
# Se cargan los archivos de excel
dfAdmitidos = pd.read_excel(routes.path_admitidos)
dfMatprimer = pd.read_excel(routes.path_matprimer)
dfInscritos = pd.read_excel(routes.path_inscritos, sheet_name='Inscritos')

In [4]:
# Se normaliza el nombre de las columnas
dfInscritos.columns = [col.replace('\n', ' ') for col in dfInscritos.columns]
dfInscritos.columns = dfInscritos.columns.str.upper()

### Agregación de Género

#### Admitidos

In [5]:
group_columns = [
    'CÓDIGO DE LA INSTITUCIÓN', 'IES PADRE',
    'INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES)', 'PRINCIPAL O SECCIONAL',
    'SECTOR IES', 'CARACTER IES', 'CÓDIGO DEL DEPARTAMENTO (IES)',
    'DEPARTAMENTO DE DOMICILIO DE LA IES', 'CÓDIGO DEL MUNICIPIO IES',
    'MUNICIPIO DE DOMICILIO DE LA IES', 'CÓDIGO SNIES DEL PROGRAMA',
    'PROGRAMA ACADÉMICO', 'NIVEL ACADÉMICO', 'NIVEL DE FORMACIÓN',
    'METODOLOGÍA', 'CÓDIGO DEL DEPARTAMENTO (PROGRAMA)',
    'DEPARTAMENTO DE OFERTA DEL PROGRAMA', 'CÓDIGO DEL MUNICIPIO (PROGRAMA)',
    'MUNICIPIO DE OFERTA DEL PROGRAMA', 'AÑO', 'SEMESTRE'   # Include AÑO to aggregate per year
]

# Group by the defined columns and sum the 'ADMITIDOS'
aggregated_dfAdmitidos = dfAdmitidos.groupby(group_columns, as_index=False)['ADMITIDOS'].sum()


#### Primer Curso

In [6]:
group_columns = [
    'CÓDIGO DE LA INSTITUCIÓN', 'IES_PADRE',
    'INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES)', 'PRINCIPAL O SECCIONAL',
    'SECTOR IES', 'CARACTER IES', 'CÓDIGO DEL DEPARTAMENTO (IES)',
    'DEPARTAMENTO DE DOMICILIO DE LA IES', 'CÓDIGO DEL MUNICIPIO (IES)',
    'MUNICIPIO DE DOMICILIO DE LA IES', 'CÓDIGO SNIES DEL PROGRAMA',
    'PROGRAMA ACADÉMICO', 'NIVEL ACADÉMICO', 'NIVEL DE FORMACIÓN',
    'METODOLOGÍA', 'CÓDIGO DEL DEPARTAMENTO (PROGRAMA)',
    'DEPARTAMENTO DE OFERTA DEL PROGRAMA', 'CÓDIGO DEL MUNICIPIO (PROGRAMA)',
    'MUNICIPIO DE OFERTA DEL PROGRAMA', 'AÑO', 'SEMESTRE'   # Include AÑO to aggregate per year
]

# Group by the defined columns and sum the 'PRIMER CURSO'
aggregated_dfMatprimer = dfMatprimer.groupby(group_columns, as_index=False)['PRIMER CURSO'].sum()

#### Inscritos

In [7]:
group_columns = [
    'CÓDIGO DE  LA INSTITUCIÓN', 'IES PADRE',
       'INSTITUCIÓN DE EDUCACIÓN SUPERIOR', 'PRINCIPAL  O  SECCIONAL',
       'SECTOR IES', 'CARACTER IES', 'CÓDIGO DEL  DEPARTAMENTO (IES)',
       'DEPARTAMENTO DE  DOMICILIO DE LA IES', 'CÓDIGO DEL  MUNICIPIO (IES)',
       'MUNICIPIO DE DOMICILIO DE LA IES', 'CÓDIGO  SNIES DEL PROGRAMA',
       'PROGRAMA ACADÉMICO', 'NIVEL ACADÉMICO', 'NIVEL DE FORMACIÓN',
       'METODOLOGÍA', 'CÓDIGO DEL  DEPARTAMENTO (PROGRAMA)',
       'DEPARTAMENTO DE OFERTA DEL PROGRAMA',
       'CÓDIGO DEL  MUNICIPIO (PROGRAMA)', 'SEMESTRE' ,'MUNICIPIO DE OFERTA DEL PROGRAMA',
       'AÑO'  # Include AÑO to aggregate per year
]

# Group by the defined columns and sum the 'ADMITIDOS'
aggregated_dfInscritos = dfInscritos.groupby(group_columns, as_index=False)['INSCRITOS'].sum()

# Creación de modelo

In [21]:
aggregated_dfMatprimer

,CÓDIGO DE LA INSTITUCIÓN,IES_PADRE,INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES),PRINCIPAL O SECCIONAL,SECTOR IES,CARACTER IES,CÓDIGO DEL DEPARTAMENTO (IES),DEPARTAMENTO DE DOMICILIO DE LA IES,CÓDIGO DEL MUNICIPIO (IES),MUNICIPIO DE DOMICILIO DE LA IES,...,NIVEL DE FORMACIÓN,METODOLOGÍA,CÓDIGO DEL DEPARTAMENTO (PROGRAMA),DEPARTAMENTO DE OFERTA DEL PROGRAMA,CÓDIGO DEL MUNICIPIO (PROGRAMA),MUNICIPIO DE OFERTA DEL PROGRAMA,AÑO,SEMESTRE,PRIMER CURSO,DATE
0,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,...,UNIVERSITARIO,PRESENCIAL,11,BOGOTA D.C.,11001,BOGOTA D.C.,2014-01-01,1,79,2014-01-01
1,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,...,UNIVERSITARIO,PRESENCIAL,11,BOGOTA D.C.,11001,BOGOTA D.C.,2014-01-01,2,78,2014-07-01
2,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,...,UNIVERSITARIO,PRESENCIAL,11,BOGOTA D.C.,11001,BOGOTA D.C.,2015-01-01,1,95,2015-01-01
3,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,...,UNIVERSITARIO,PRESENCIAL,11,BOGOTA D.C.,11001,BOGOTA D.C.,2015-01-01,2,119,2015-07-01
4,1101,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,...,UNIVERSITARIO,PRESENCIAL,11,BOGOTA D.C.,11001,BOGOTA D.C.,2014-01-01,1,41,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212255,9936,9936,CORPORACIÓN UNIVERSITARIA AUTÓNOMA DEL NORTE,Principal,PRIVADA,Institución Universitaria/Escuela Tecnológica,54,Norte de Santander,54001,San José de Cúcuta,...,Especialización universitaria,Presencial,54,Norte de Santander,54001,San José de Cúcuta,2022-01-01,1,5,2022-01-01
212256,9936,9936,CORPORACIÓN UNIVERSITARIA AUTÓNOMA DEL NORTE,Principal,PRIVADA,Institución Universitaria/Escuela Tecnológica,54,Norte de Santander,54001,San José de Cúcuta,...,Universitario,Presencial,54,Norte de Santander,54001,San José de Cúcuta,2022-01-01,1,9,2022-01-01
212257,9936,9936,CORPORACIÓN UNIVERSITARIA AUTÓNOMA DEL NORTE,Principal,PRIVADA,Institución Universitaria/Escuela Tecnológica,54,Norte de Santander,54001,San José de Cúcuta,...,Universitario,Presencial,54,Norte de Santander,54001,San José de Cúcuta,2022-01-01,2,7,2022-07-01
212258,9936,9936,CORPORACIÓN UNIVERSITARIA AUTÓNOMA DEL NORTE,Principal,PRIVADA,Institución Universitaria/Escuela Tecnológica,54,Norte de Santander,54001,San José de Cúcuta,...,Universitario,Presencial,54,Norte de Santander,54001,San José de Cúcuta,2022-01-01,1,34,2022-01-01


### Preparación del dataframe

In [9]:
aggregated_dfMatprimer['AÑO'] = pd.to_datetime(aggregated_dfMatprimer['AÑO'], format='%Y')
aggregated_dfMatprimer['DATE'] = aggregated_dfMatprimer.apply(
    lambda row: row['AÑO'] + pd.DateOffset(months=6*(row['SEMESTRE']-1)),
    axis=1
)
aggregated_dfMatprimer['INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES)']

In [44]:
# Aggregate the 'PRIMER CURSO' per year and program
program_df = aggregated_dfMatprimer.groupby(['DATE', 'PROGRAMA ACADÉMICO', 'INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES)']).agg({'PRIMER CURSO': 'sum'}).reset_index()


In [45]:
program_df

,DATE,PROGRAMA ACADÉMICO,INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES),PRIMER CURSO
0,2014-01-01,ACTIVIDAD FISICA Y DEPORTE,FUNDACION UNIVERSITARIA LUIS AMIGO FUNLAM,52
1,2014-01-01,ACUICULTURA,UNIVERSIDAD DE CORDOBA,56
2,2014-01-01,ADMINISTRACION,ESCUELA NAVAL DE CADETES ALMIRANTE PADILLA,20
3,2014-01-01,ADMINISTRACION FINANCIERA,FUNDACION UNIVERSITARIA DEL AREA ANDINA,11
4,2014-01-01,ADMINISTRACION FINANCIERA,UNIVERSIDAD DE SANTANDER - UDES,25
...,...,...,...,...
127875,2022-07-01,ZOOTECNIA,UNIVERSIDAD INDUSTRIAL DE SANTANDER,16
127876,2022-07-01,ZOOTECNIA,UNIVERSIDAD LIBRE,1
127877,2022-07-01,ZOOTECNIA,UNIVERSIDAD NACIONAL ABIERTA Y A DISTANCIA UNAD,463
127878,2022-07-01,ZOOTECNIA,UNIVERSIDAD NACIONAL DE COLOMBIA,152


### Holts-Winters

In [12]:
# Fit model
model = ExponentialSmoothing(program_df['y'], trend="add", seasonal="add", seasonal_periods=2)
model_fit = model.fit()

# Forecast next few periods
forecast = model_fit.forecast(steps=4)
print(forecast)

18    48.347222
19    53.347222
20    48.238889
21    53.238888
dtype: float64


c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [13]:
n_splits = 3  # for simplicity, assuming we want 3 validation points

# Initialize lists to store the performance of each fold
mse_scores = []
mae_scores = []

# The minimum number of observations we decide is needed to start forecasting
min_train_size = 6

for t in range(min_train_size, len(program_df) - n_splits + 1):
    train = program_df.iloc[:t]
    test = program_df.iloc[t:t+n_splits]  # here n_splits is 1 for one-step ahead prediction
    
    # Fit model
    model = ExponentialSmoothing(program_df['y'], seasonal='add', seasonal_periods=4)
    model_fit = model.fit()
    
    # Forecast
    forecast = model_fit.forecast(steps=n_splits)
    
    # Calculate performance metrics
    mse = mean_squared_error(test['y'], forecast)
    mae = mean_absolute_error(test['y'], forecast)
    
    # Store the scores
    mse_scores.append(mse)
    mae_scores.append(mae)

# Print average scores
print(f'Average MSE: {np.mean(mse_scores)}')
print(f'Average MAE: {np.mean(mae_scores)}')

c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(

Average MSE: 38.6275002887143
Average MAE: 4.95999997427646


c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JUAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future

### Modelo Prophet

In [94]:
# Assuming `aggregated_dfMatprimer` is already loaded with appropriate data
# Aggregate the 'PRIMER CURSO' per year and program
program_df = aggregated_dfMatprimer.groupby(['DATE', 'PROGRAMA ACADÉMICO', 'INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES)']).agg({'PRIMER CURSO': 'sum'}).reset_index()

# Specify the program you want to forecast
target_program = "ADMINISTRACION FINANCIERA"

# Filter data for the specific program
specific_program_data = program_df[program_df['PROGRAMA ACADÉMICO'] == target_program]

# Convert 'INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES)' to one-hot encoding
one_hot_institutions = pd.get_dummies(specific_program_data['INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES)'])
specific_program_data = pd.concat([specific_program_data, one_hot_institutions], axis=1)

# Initialize the Prophet model with additional regressors for each institution
model = Prophet()

# Add institution one-hot encoding as additional regressors
for column in one_hot_institutions.columns:
    model.add_regressor(column)

# Prepare the data for Prophet
specific_program_data['ds'] = specific_program_data['DATE']
specific_program_data['y'] = specific_program_data['PRIMER CURSO']

# Create and fit the Prophet model using all the data and institution regressors
model.fit(specific_program_data[['ds', 'y'] + list(one_hot_institutions.columns)])

# Make future predictions
future = model.make_future_dataframe(periods=4, freq='6M')

# Identify the last historical date in the original dataset
last_hist_date = specific_program_data['ds'].max()

# Adjust only the future prediction dates to start at the beginning of a month
future.loc[future['ds'] > last_hist_date, 'ds'] = future['ds'][future['ds'] > last_hist_date] + pd.offsets.MonthBegin(-1)

# Carry over the one-hot encoding for institutions to the future DataFrame
for column in one_hot_institutions.columns:
    future[column] = 0  # or appropriate values based on expected future conditions

# Predict
forecast = model.predict(future)

# Perform cross-validation
df_cv = cross_validation(model, initial='1460 days', period='182 days', horizon='730 days')

# Calculate the performance metrics
df_p = performance_metrics(df_cv)

# Show the performance metrics
print(df_p)

# Display the first few rows of the forecast results
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Plotting with Plotly
trace1 = go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Prediction')
trace2 = go.Scatter(x=specific_program_data['ds'], y=specific_program_data['y'], mode='lines+markers', name='Actual Data')
layout = go.Layout(title='Academic Program Demand Forecast', xaxis_title='Date', yaxis_title='Quantity')
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

17:51:11 - cmdstanpy - INFO - Chain [1] start processing
17:51:11 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/6 [00:00<?, ?it/s]

17:51:11 - cmdstanpy - INFO - Chain [1] start processing
17:51:11 - cmdstanpy - INFO - Chain [1] done processing
17:51:11 - cmdstanpy - INFO - Chain [1] start processing
17:51:11 - cmdstanpy - INFO - Chain [1] done processing
17:51:11 - cmdstanpy - INFO - Chain [1] start processing
17:51:11 - cmdstanpy - INFO - Chain [1] done processing
17:51:12 - cmdstanpy - INFO - Chain [1] start processing
17:51:12 - cmdstanpy - INFO - Chain [1] done processing
17:51:12 - cmdstanpy - INFO - Chain [1] start processing
17:51:12 - cmdstanpy - INFO - Chain [1] done processing
17:51:12 - cmdstanpy - INFO - Chain [1] start processing
17:51:12 - cmdstanpy - INFO - Chain [1] done processing


    horizon           mse        rmse        mae      mape     mdape  \
0  181 days  16949.045862  130.188501  76.275174  3.121098  0.886351   
1  182 days  21249.554688  145.772270  76.835011  3.839116  0.740524   
2  184 days  28370.074677  168.434185  81.666608  3.540060  0.605607   
3  362 days  25005.274481  158.130562  79.587881  3.386650  0.656724   
4  363 days  19823.485455  140.795900  77.047081  2.764657  0.936557   
5  364 days  21926.046306  148.074462  81.668278  4.231163  0.710699   
6  365 days  20667.914895  143.763399  79.591529  3.662553  0.754237   
7  366 days  27499.089459  165.828494  85.498058  4.379012  0.880824   
8  544 days  24931.672190  157.897664  82.901434  5.243453  0.850961   
9  546 days  21031.931544  145.023900  83.391571  5.254276  0.925551   
10 547 days  21881.071082  147.922517  84.682243  5.538102  0.893292   
11 548 days  28031.125307  167.424984  86.391959  4.408337  0.822235   
12 549 days  35782.869358  189.163605  90.709180  7.185184  0.63

### Regresión Lineal

In [ ]:
linear_model = LinearRegression()

R^2 